# Populated Places of BC

Goal: Create a list of places in BC in .csv format which have a population of at least 1. Includes Cities, Towns, Villages, Reserves and other populated regions.

Combines data from:
- Geographical Place Names in BC
- 2021 Population Census of BC communities

*Note that the original CSVs have been modified by hand to remove metadata information (headers & footnotes) and aggregation rows*

In [ ]:
import pandas as pd

features = pd.read_csv("./geographical_names_bc.csv.gz", compression="gzip")
populations = pd.read_csv("./population_census_bc.csv.gz", compression="gzip")

features.drop(
  columns=[
    "ISO Language Code",
    "Concise Code",
    "Toponymic Feature ID",
    "Province - Territory",
    "Source",
    "Relevance at Scale",
    "Syllabic Form",
    "Decision Date",
    "CGNDB ID",
    "Language"
  ],
  inplace=True
)

populations.drop(
  columns=[
    "Population, 2016",
    "Population percentage change, 2016 to 2021",
    "Total private dwellings, 2021",
    "Total private dwellings, 2016",
    "Total private dwellings percentage change, 2016 to 2021",
    "Private dwellings occupied by usual residents, 2021",
    "Private dwellings occupied by usual residents, 2016",
    "Private dwellings occupied by usual residents percentage change, 2016 to 2021",
    "National population rank, 2021",
    "Province/territory population rank, 2021",
    "Geographic area type abbreviation"
  ],
  inplace=True
)

populations.rename(
  columns={
    "Geographic name": "Geographical Name"
  },
  inplace=True
)

# Only keep columns in features where a population can be found
joined = features.join(populations.set_index("Geographical Name"), on="Geographical Name", how="left")

joined = joined[
  (joined["Population, 2021"].notnull()) &
  (joined["Land area in square kilometres, 2021"].notnull()) &
  (joined["Population density per square kilometre, 2021"].notnull())
]

# Replace numeric columns in "123,456" string format with integers
joined['Population, 2021'] = joined['Population, 2021'].str.replace(",", "").astype(int)
joined['Land area in square kilometres, 2021'] = joined['Land area in square kilometres, 2021'].str.replace(",", "").astype(float)
joined['Population density per square kilometre, 2021'] = joined['Population density per square kilometre, 2021'].str.replace(",", "").astype(float)


joined = joined[(joined['Population, 2021'] > 0)]

# Remove duplicate features (gets rid of, for example "Bowen Island" as an island, and "Bown Island" as a municipality)
joined = joined.drop_duplicates(subset=["Geographical Name"])

# Create an UID for each populated place
joined["id"] = joined.index + 1

joined.to_csv("../BCPopulatedPlaces.csv", index=False)